## PARA EXTRAER SALDO Y PROGRAMACION FINANCIERA DEL F12B https://ofi5.mef.gob.pe/inviertews/Repseguim/ResumF12B?codigo=

In [ ]:
!pip install selenium webdriver-manager openpyxl pandas

In [ ]:
!pip install playwright
!playwright install chromium

In [1]:
import pandas as pd
import time
import os
import random
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as Wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ================================================================
# CONFIGURACIÓN OPTIMIZADA
# ================================================================
RUTA_ENTRADA = r"C:\Users\user\Downloads\mef1.xlsx"
RUTA_SALIDA = r"C:\Users\user\Downloads\SALDO Y PROGRAMACION CUI 3507.xlsx"
MAX_REINTENTOS = 2
MODO_VISIBLE = True
GUARDAR_CADA = 5

# Timeouts optimizados
TIMEOUT_PAGINA = 20
TIMEOUT_ELEMENTO = 10
ESPERA_ENTRE_CUIS = (1.5, 3)  # Espera aleatoria anti-saturación

# ================================================================
# VERIFICACIÓN INICIAL
# ================================================================
print("🔍 Verificando configuración...")
print(f"📂 Entrada: {RUTA_ENTRADA}")
print(f"   {'✅ Existe' if os.path.exists(RUTA_ENTRADA) else '❌ NO EXISTE'}")
print(f"💾 Salida: {RUTA_SALIDA}\n")

# ================================================================
# FUNCIONES DE CHECKPOINT
# ================================================================
def cargar_progreso():
    """Carga progreso previo si existe"""
    if os.path.exists(RUTA_SALIDA):
        try:
            df = pd.read_excel(RUTA_SALIDA)
            print(f"📥 Progreso encontrado: {len(df)} CUIs ya procesados")
            return df.to_dict('records')
        except Exception as e:
            print(f"⚠️ Error cargando progreso: {e}")
    return []

def obtener_pendientes(completa, procesados):
    """Calcula CUIs pendientes"""
    if not procesados:
        return completa
    cuis_ok = {str(r['CUI']) for r in procesados}
    pendientes = [cui for cui in completa if str(cui) not in cuis_ok]
    if pendientes:
        print(f"⏳ Pendientes: {len(pendientes)}")
    return pendientes

def guardar(resultados):
    """Guarda progreso"""
    try:
        pd.DataFrame(resultados).to_excel(RUTA_SALIDA, index=False)
        return True
    except Exception as e:
        print(f"⚠️ Error guardando: {e}")
        return False

# ================================================================
# LEER CUIs
# ================================================================
print("📂 Cargando CUIs...")
try:
    df_cui = pd.read_excel(RUTA_ENTRADA)
    
    if 'CUI' in df_cui.columns:
        lista_completa = df_cui['CUI'].astype(str).tolist()
    elif 'CODIGO' in df_cui.columns:
        lista_completa = df_cui['CODIGO'].astype(str).tolist()
    else:
        print(f"❌ Error: No se encontró columna 'CUI' o 'CODIGO'")
        print(f"📋 Columnas disponibles: {', '.join(df_cui.columns)}")
        exit()
    
    print(f"✅ {len(lista_completa)} CUIs cargados")
except Exception as e:
    print(f"❌ Error leyendo archivo: {e}")
    exit()

resultados = cargar_progreso()
lista_cuis = obtener_pendientes(lista_completa, resultados)

if not lista_cuis:
    print("🎉 ¡Todos los CUIs ya fueron procesados!")
    exit()

# ================================================================
# CONFIGURAR NAVEGADOR OPTIMIZADO
# ================================================================
def crear_driver():
    service = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()

    if not MODO_VISIBLE:
        options.add_argument("--headless=new")
    else:
        options.add_argument("--start-maximized")

    # Optimizaciones de rendimiento
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-logging")
    options.add_argument("--log-level=3")
    options.add_argument("--silent")

    # Deshabilitar recursos innecesarios
    prefs = {
        "profile.managed_default_content_settings.images": 2,  # Sin imágenes
        "profile.default_content_setting_values.notifications": 2,
        "profile.default_content_setting_values.media_stream": 2,
    }
    options.add_experimental_option("prefs", prefs)
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.page_load_strategy = 'normal'

    driver = Chrome(service=service, options=options)
    driver.set_page_load_timeout(TIMEOUT_PAGINA)
    driver.set_script_timeout(15)
    
    return driver

driver = crear_driver()
print(f"⚡ Procesando {len(lista_cuis)} CUIs pendientes\n")

# ================================================================
# EXTRACCIÓN DE DATOS - SOLO 2 CAMPOS
# ================================================================
def extraer_saldo_programacion(driver):
    """Extrae SOLO Programación Financiera y Déficit/Saldo"""
    try:
        # Esperar que cargue uno de los elementos
        Wait(driver, TIMEOUT_ELEMENTO).until(
            EC.presence_of_element_located((By.ID, "prfin"))
        )
        
        # Pequeña espera para asegurar que todo cargó
        time.sleep(0.8)
        
        # Extraer los 2 valores con JavaScript
        datos = driver.execute_script("""
            function getText(id) {
                try {
                    var elem = document.getElementById(id);
                    if (!elem) return 'NO DISPONIBLE';
                    var texto = (elem.textContent || elem.innerText || '').trim();
                    return texto || 'NO DISPONIBLE';
                } catch(e) {
                    return 'NO DISPONIBLE';
                }
            }
            
            return {
                programacion_financiera: getText('prfin'),
                deficit_saldo: getText('defsal')
            };
        """)

        # Validar que al menos uno tenga datos
        if (datos['programacion_financiera'] == 'NO DISPONIBLE' and 
            datos['deficit_saldo'] == 'NO DISPONIBLE'):
            return None
            
        return datos

    except TimeoutException:
        print(f"⏱️", end=" ")
        return None
    except Exception as e:
        print(f"⚠️", end=" ")
        return None

# ================================================================
# PROCESAR CUI
# ================================================================
def procesar_cui(cui, intento=1):
    try:
        # URL directa
        url = f"https://ofi5.mef.gob.pe/inviertews/Repseguim/ResumF12B?codigo={cui}"
        driver.get(url)
        
        # Espera mínima
        time.sleep(0.5)
        
        # Extraer datos
        datos = extraer_saldo_programacion(driver)
        
        if not datos:
            raise Exception("No se pudieron extraer los datos")

        return datos

    except Exception as e:
        if intento < MAX_REINTENTOS:
            print(f"🔄{intento+1}", end=" ")
            time.sleep(1.5)
            return procesar_cui(cui, intento + 1)
        else:
            raise e

# ================================================================
# LOOP PRINCIPAL
# ================================================================
tiempo_inicio = time.time()
exitosos = 0
fallos = 0

print(f"{'='*65}")
print(f"🚀 INICIO: {time.strftime('%H:%M:%S')}")
print(f"{'='*65}\n")

try:
    for idx, cui in enumerate(lista_cuis, 1):
        t_inicio = time.time()
        print(f"[{idx}/{len(lista_cuis)}] {cui}: ", end="")
        
        try:
            datos = procesar_cui(cui)
            
            resultados.append({
                "CUI": cui,
                "Programación Financiera Actualizada": datos['programacion_financiera'],
                "Déficit o Saldo": datos['deficit_saldo']
            })
            exitosos += 1
            print(f"✅ ({time.time()-t_inicio:.1f}s)")
            
        except Exception as e:
            resultados.append({
                "CUI": cui,
                "Programación Financiera Actualizada": "NO DISPONIBLE",
                "Déficit o Saldo": "NO DISPONIBLE"
            })
            fallos += 1
            print(f"❌ ({time.time()-t_inicio:.1f}s)")

        # GUARDAR CADA N CUIs
        if (idx % GUARDAR_CADA == 0) or (idx == len(lista_cuis)):
            if guardar(resultados):
                if idx % GUARDAR_CADA == 0:
                    print(f"   💾 Guardado")

        # ESPERA ANTI-SATURACIÓN
        if idx < len(lista_cuis):
            espera = random.uniform(ESPERA_ENTRE_CUIS[0], ESPERA_ENTRE_CUIS[1])
            time.sleep(espera)

        # Progreso cada 10 CUIs
        if idx % 10 == 0:
            t = time.time() - tiempo_inicio
            prom = t / idx
            eta = (len(lista_cuis) - idx) * prom
            tasa = idx / t * 60  # CUIs por minuto
            print(f"   📊 {exitosos}✅ {fallos}❌ | {t:.0f}s | ETA: {eta/60:.1f}min | {tasa:.1f} CUI/min\n")

except KeyboardInterrupt:
    print("\n\n⚠️ INTERRUMPIDO")
    guardar(resultados)
    driver.quit()
    print(f"💾 Progreso guardado en: {RUTA_SALIDA}")
    print(f"📊 Ejecuta nuevamente para continuar.")
    exit()

# ================================================================
# FINALIZAR
# ================================================================
driver.quit()
guardar(resultados)

tiempo_total = time.time() - tiempo_inicio

print(f"\n{'='*65}")
print("🏁 COMPLETADO")
print(f"{'='*65}")
print(f"📊 Total procesados: {len(lista_cuis)}")
print(f"✅ Exitosos: {exitosos}")
print(f"❌ Fallidos: {fallos}")
print(f"⏱️ Tiempo total: {tiempo_total/60:.1f} minutos")
print(f"⚡ Velocidad: {len(lista_cuis)/tiempo_total*60:.1f} CUI/min")
print(f"💾 Archivo guardado en:")
print(f"   {RUTA_SALIDA}")
print(f"{'='*65}")

# Verificar archivo
if os.path.exists(RUTA_SALIDA):
    print(f"✅ Archivo confirmado: {os.path.getsize(RUTA_SALIDA)/1024:.1f} KB")
    
    # Preview de resultados
    if resultados:
        df_preview = pd.DataFrame(resultados[-5:])
        print("\n📋 PREVIEW (últimos 5 registros):")
        print(df_preview.to_string(index=False))
else:
    print(f"⚠️ ADVERTENCIA: No se pudo confirmar el archivo")

🔍 Verificando configuración...
📂 Entrada: C:\Users\user\Downloads\mef1.xlsx
   ✅ Existe
💾 Salida: C:\Users\user\Downloads\SALDO Y PROGRAMACION CUI 3507.xlsx

📂 Cargando CUIs...
✅ 3509 CUIs cargados
📥 Progreso encontrado: 3320 CUIs ya procesados
⏳ Pendientes: 189
⚡ Procesando 189 CUIs pendientes

🚀 INICIO: 20:11:51

[1/189] 2519054: ✅ (2.6s)
[2/189] 2522145: 🔄2 ❌ (4.4s)
[3/189] 2522146: 🔄2 ❌ (4.3s)
[4/189] 2523988: ✅ (2.0s)
[5/189] 2524757: ✅ (1.9s)
   💾 Guardado
[6/189] 2534905: ✅ (1.9s)
[7/189] 2539877: ✅ (1.9s)
[8/189] 2540062: ✅ (2.0s)
[9/189] 2540068: ✅ (2.0s)
[10/189] 2540133: ✅ (2.0s)
   💾 Guardado
   📊 8✅ 2❌ | 48s | ETA: 14.4min | 12.4 CUI/min

[11/189] 2540136: ✅ (1.9s)
[12/189] 2540759: ✅ (2.1s)
[13/189] 2540767: ✅ (1.9s)
[14/189] 2545079: ✅ (1.9s)
[15/189] 2549354: ✅ (2.1s)
   💾 Guardado
[16/189] 2549677: ✅ (2.0s)
[17/189] 2555285: ✅ (1.9s)
[18/189] 2556532: 🔄2 ❌ (4.3s)
[19/189] 2556541: 🔄2 ❌ (4.3s)
[20/189] 2556636: ✅ (2.0s)
   💾 Guardado
   📊 16✅ 4❌ | 96s | ETA: 13.5min | 1